In [1]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `K` was defined in the following phases: ['FCC_A1', 'FCC_L12'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `S` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
C:\Users\Dmitry\AppData\Roaming\Python\Python37\site-packages\pycalphad\io\tdb.py:925: UserWarning: The type definition character `P` was defined in the following phases: ['FCC_A1', 'FCC_4SL', 'HCP_A3', 'HCP_4SL'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(

In [2]:
# constants:
P = 101325

# Компоненты и фазы

In [3]:
import random

random.seed(10)
all_components = list(db.elements)
all_phases =  list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


Количество фаз =2023
Количество компонент =80
['AG', 'AL', 'AM', 'AS', 'AU', 'B', 'BA', 'BE', 'BI', 'C', 'CA', 'CD', 'CE', 'CO', 'CR', 'CS', 'CU', 'DY', 'ER', 'EU', 'FE', 'GA', 'GD', 'GE', 'H', 'HF', 'HG', 'HO', 'IN', 'IR', 'K', 'LA', 'LI', 'LU', 'MG', 'MN', 'MO', 'N', 'NA', 'NB', 'ND', 'NI', 'NP', 'O', 'OS', 'P', 'PA', 'PB', 'PD', 'PR', 'PT', 'PU', 'RB', 'RE', 'RH', 'RU', 'S', 'SB', 'SC', 'SE', 'SI', 'SM', 'SN', 'SR', 'TA', 'TB', 'TC', 'TE', 'TH', 'TI', 'TL', 'TM', 'U', 'V', 'VA', 'W', 'Y', 'YB', 'ZN', 'ZR']
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']


# Бинарные фазы для компонент

In [4]:
counter = 0 
possible_phases = []

for phase in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints    
    found_sublattices = 0
    for sublattice_c in phase[1].constituents:
        found = False
        for comp in components:
            if len([species.name for species in sublattice_c if species.name == comp]) > 0:
                found = True
                break
        if found:
            found_sublattices += 1
        else:
            break
    
    if found_sublattices == len(phase[1].constituents):
        possible_phases.append(phase[0])
        
possible_phases.sort()
print(possible_phases)

['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 'RHOMBO_C19', 'SIGMA', 'TETRAGONAL_A6',

In [5]:
import numpy as np

def getPhases(int_mask, all_phases = all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]
def phasesToMask(phases, all_phases = all_phases):
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)
def phaseToMask(phase, all_phases = all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

phases_mask = phasesToMask(possible_phases)
print(possible_phases == getPhases(phases_mask, all_phases))

True


# Расчет фазовых равновесий

In [6]:
skip_cond_component = 'V'
temp = 1200
conditions = {v.T:temp,v.P:P}
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']
# TODO: make random amount of component
amount = 1.0/len(components)

for comp in components:
    if comp != skip_cond_component:
        conditions[v.X(comp)] = amount

calc_components = components
calc_components.append('VA')
print(conditions)
print(calc_components)
eq = equilibrium(db,calc_components,getPhases(phases_mask),conditions, verbose=False)
# gibbs_energy = eq.GM

{T: 1200, P: 101325, X_AU: 0.125, X_RH: 0.125, X_SM: 0.125, X_ZR: 0.125, X_AL: 0.125, X_HG: 0.125, X_SE: 0.125}
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE', 'VA']


In [7]:
def flatten_array(arr):
    result = []
    for item in arr.flat:
        if isinstance(item, np.ndarray):
            result.extend(flatten_array(item))
        else:
            result.append(item)
    return result

In [8]:
eq_phases = flatten_array(np.array(eq.Phase))
print(eq_phases)
eq_phases_bin = [phaseToMask(phase) for phase in eq_phases]
print([getPhases(phase) for phase in eq_phases_bin])

['AUZR', 'LIQUID', 'AL2SM', '', '', '', '', '', '']
[['AUZR'], ['LIQUID'], ['AL2SM'], [], [], [], [], [], []]


# Функции

In [9]:
from pycalphad.core.utils import instantiate_models
# instantiate_models(db, components, getAllPhases(components_phases))
print(getPhases(phases_mask))

['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 'RHOMBO_C19', 'SIGMA', 'TETRAGONAL_A6',

In [49]:
from symengine import Piecewise, Symbol, lib
import math
import sympy

def parseMul(mul):
    key = ""
    value = 0
    for arg in mul:
        if isinstance(arg, float):
            value = arg
        else:
            key = str(arg)
    return (key, value)
        

def parsePiecewise(expr, temp):
    argument = 0
    for arg in expr.args_as_sympy():
        if arg.is_Add:
            argument = arg.as_coeff_Mul()
        if arg.is_Boolean and arg.subs(T,temp):
            break
    if argument == 0:
        print(f"Failed to parse:{expr}")
    print(argument)

    parsed_dict = {}

    for arg in argument[1].args:
        ordered_arg = arg.as_ordered_factors()
        if len(ordered_arg) == 1:
           parsed_dict["T**0"] = ordered_arg[0]
           continue
        elif len(ordered_arg) == 3:           
            key, value = parseMul(ordered_arg)
            parsed_dict[key] = value
    
    return parsed_dict

temp = 1200
T = Symbol('T')
expr = db.symbols['GBCCGD']
parsePiecewise(expr, temp)

(1, -22038836.0/T**1.0 + 180.097094*T*log(T) - 1349.588729*T - 0.119550229*T**2.0 + 1.17915728e-5*T**3.0 + 152792.74)


{'T**0': 152792.740000000, 'T**(-1.0)': 0, 'T': 0, 'T**2.0': 0, 'log(T)': 0}

## Парсер функций

In [73]:
from pycalphad.core.utils import instantiate_models
models = instantiate_models(db, calc_components, getPhases(phases_mask))

In [74]:

temp = 1200
T = Symbol('T')

print(models['AUZR'].ast.subs(T, temp).extract_coeffs())
# print(sympy.Poly(models['AUZR'], temp).coeffs()[0])
# models['AUZR'].args


AttributeError: 'Add' object has no attribute 'extract_coeffs'

In [ ]:
import sympy

temp = 1200
T = sympy.Symbol('T')

piecewise_expr = sympy.Piecewise(
    (2*T + 3, T < 1000),
    (3*T + 1, T >= 1000)
)

coeffs = []
for expr, cond in piecewise_expr.args:
    if cond.subs(T, temp):
        polynomial_expr = sympy.simplify(expr.subs(T, temp))
        expr_coeffs = polynomial_expr.extract_coeffs()
        coeffs.extend(expr_coeffs)

print(coeffs)


AttributeError: 'Integer' object has no attribute 'extract_coeffs'

In [ ]:
for model in models:
    print(model)
    print(models[model].ast)
    break

In [ ]:
import sympy

def get_piecewise_true_arg(piecewise_func, x_val):
    """
    Returns the true argument of piecewise_func for x = x_val. Assumes piecewise_func is of the form Piecewise((expr1, cond1), (expr2, cond2), ...).
    """
    x = sympy.Symbol('x')
    
    for expr, cond in piecewise_func.args:
        if cond.subs(x, x_val):
            print(x, x_val)
            return expr.subs(x, x_val)
    
    return None

# example usage
piecewise_func = sympy.Piecewise(
    (x**2, x < 0),
    (x, x >= 0)
)
true_arg = get_piecewise_true_arg(piecewise_func,-10)
print(true_arg)  # output: 1


x -10
100


In [ ]:
from sympy import Piecewise, symbols
 # Define the piecewise function
x = symbols('x')
f = Piecewise((x**2, x < 0), (x, x >= 0))
 # Specify the value of x
x_value = 3
 # Check which of the conditions are true for the value of x
for i in range(len(f.args)):
    condition = f.args[i][1]
    if condition.subs(x, x_value):
        print(f.args[i][0])

x


# Текущие результаты:

In [ ]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [ ]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [ ]:
print('Все фазы для компонентов из списка:')
print(len(getPhases(phases_mask)))
print(getPhases(phases_mask))

Все фазы для компонентов из списка:
93
['AGSB_ORTHO', 'AL11SM3_HT', 'AL11SM3_LT', 'AL21V2', 'AL23V4', 'AL2PD5', 'AL2SM', 'AL2ZR', 'AL2ZR3', 'AL3M_D022', 'AL3NB', 'AL3PD2_D513', 'AL3SM', 'AL3ZR', 'AL3ZR2', 'AL3ZR5', 'AL45V7', 'AL4RE_LT', 'AL4ZR5', 'AL5FE4', 'AL8V5', 'ALCE_AMORPHOUS', 'ALCR2_C11B', 'ALCU_THETA', 'ALM_D019', 'ALNB3', 'ALPD2_C37', 'ALPHA_PU', 'ALPT3', 'ALRE2_C11B', 'ALSM', 'ALSM2', 'ALTA_SIGMA', 'ALTI', 'ALZR', 'ALZR2', 'ALZR3', 'AU10ZR7', 'AU2HF', 'AU2ZR', 'AU2ZR3', 'AU3ZR', 'AU4ZR', 'AU51ND14', 'AU5HF', 'AUHF_ALPHA', 'AUHF_BETA', 'AUZN_BETA', 'AUZR', 'AUZR2', 'AUZR3', 'BCT_A5', 'CU2SE_ALPHA', 'CU2SE_BETA', 'DIAMOND_A4', 'D_GAMMA', 'FEUZR_DELTA', 'FEZR2', 'FEZR3', 'GA5V6', 'GA7V6_D82', 'GAMMA_PU', 'GAV3_A15', 'HEXAGONAL_A8', 'IR3ZR', 'IRZR_ALPHA', 'IRZR_BETA', 'LAVES_C14', 'LAVES_C15', 'LAVES_C36', 'LIQUID', 'MB2_C32', 'MN2ZR', 'MSM_A', 'MSM_B', 'NI10ZR7', 'NI3ZR', 'NI5ZR', 'OMEGA_ZR', 'ORTHORHOMBIC_A20', 'PD3ZR', 'PT2V', 'PT3V_D022', 'PTV3_A15', 'PTV_B19', 'RHOMBO_A10', 

In [ ]:
print(f'Все фазы для компонентов из списка(бинарные): {bin(phases_mask)}')

Все фазы для компонентов из списка(бинарные): 0b1000010000000000000001000000000000000001100000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000001100000000000000011000000000000000000001000000000100000000000000000000000000000000000000000000010000000000000000000000000000000000000110000000000000000000000000000000000100000000000001000000000000000000000000000000000000000100000000000000000000000000000000000000000000001100000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000010000000000000000000000100000000000000000000000000111000000000000000000000000000000001100010000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000001000000100000001000000000000000000000000000000000110000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
print('Параметры расчета:')
print(f'Давление {P}, Температура {temp}, Количестов вещества по {amount} для компоненты')

Параметры расчета:
Давление 101325, Температура 1200, Количестов вещества по 0.125 для компоненты


In [ ]:
print('Значения функций для компонентов:')
print(function_value)

Значения функций для компонентов:
OrderedDict([('GB', None), ('GBCCZR', -69104.2656226965), ('GBCCCZ', -5725.0), ('GBCCAU', 2930.0 + GHSERAU), ('GLAVAL', 5000.0 + GHSERAL), ('GHCPAU', 2160.75 + GHSERAU), ('GBCCL', None), ('GHSERSE', 0), ('GHSERAU', -77272.6856248379), ('GBCCMZ', 0.0), ('GFCCGD', 500.0 + GHSERGD), ('GHSERAL', -54976.3276402473), ('GHSERHG', -112210.200050902), ('GFE', None), ('GLIQSE', 0), ('GBCCAL', 4307.4 + GHSERAL), ('GHSERGD', -105270.77857148), ('GBCCGD', -104573.24219628), ('GLIQGD', -102198.298584263), ('GHSERRH', 0), ('GHCPRH', 2400.0 + GHSERRH), ('GHCPAL', 3321.0 + GHSERAL), ('GLIQZR', 7251.2987626752 + GHSERZR), ('GHSERZR', -68894.6050887857), ('GFCCHG', 10046.6 + GHSERHG), ('GHSERV', -57463.158455921), ('GLIQAL', -57996.2850123662), ('GRHOMBHG', -104149.498678239), ('GFENI', None), ('GFCCZR', 6520.0 + GHSERZR), ('U', None), ('GHSERVZ', -57463.158455921), ('GLIQV', 9415.59444185221 + GHSERV), ('LB', None), ('GLIQRH', -48315.478673478), ('GHCPHG', -103109.49867

In [ ]:
print('Фазы для заданых параметров:')
print(eq_phases)
print('В бинарном виде:')
print([bin(phase) for phase in eq_phases_bin])

Фазы для заданых параметров:
['AUZR', 'LIQUID', 'AL2SM', '', '', '', '', '', '']
В бинарном виде:
['0b10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', '0b10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000